In [23]:
# mounts Google Drive to the Colab VM
from google.colab import drive, files
drive.mount('/content/drive')

# project github foldername
FOLDERNAME = '/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/'

# load python files from foldername
import sys
sys.path.append(FOLDERNAME)

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2

Mounted at /content/drive
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import xarray as xr
import pandas as pd
import pickle
import datetime 
import numpy as np

!sudo apt-get install libeccodes0

!pip install cfgrib
!pip install netcdf4

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libeccodes-data
The following NEW packages will be installed:
  libeccodes-data libeccodes0
0 upgraded, 2 newly installed, 0 to remove and 32 not upgraded.
Need to get 2,861 kB of archives.
After this operation, 66.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libeccodes-data all 2.6.0-2 [1,260 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libeccodes0 amd64 2.6.0-2 [1,601 kB]
Fetched 2,861 kB in 1s (2,954 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tt

In [0]:
directory = '/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/'
level = '2m'
model = 'arpege'
zone = "NW"
date = datetime.datetime(2017, 8, 26, 0,0)
directory_2 = directory + 'models_2D_' + zone + '/'
# fname = directory_2 + f'{model}/{level}/{model}_{level}_{zone}_{date.year}{str(date.month).zfill(2)}{str(date.day).zfill(2)}000000.grib'
# data = xr.open_dataset(fname, engine='cfgrib') 
fname = directory_2 + f'{model}/{level}/{model}_{level}_{zone}_{date.year}{str(date.month).zfill(2)}{str(date.day).zfill(2)}000000.nc'
data = xr.open_dataset(fname) 

In [158]:
directory = "/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/"
date = datetime.datetime(2017, 8, 26, 0,0)

level = '2m'
model = 'arpege'
zone = "NW"
directory_2 = directory + 'models_2D_' + zone + '/'

if level == '2m':
  params = ['t2m', 'd2m', 'r']
elif level == '10m':
  params = ['ws', 'p3031', 'u10', 'v10']
elif level == 'P_sea_level':
  params = ['msl']
else:
  params = ['tp']

date_prev = date + datetime.timedelta(days = -1)
date_next = date + datetime.timedelta(days = 1)


fname_prev = directory_2 + f'{model}/{level}/{model}_{level}_{zone}_{date_prev.year}{str(date_prev.month).zfill(2)}{str(date_prev.day).zfill(2)}000000.grib'
data_prev = xr.open_dataset(fname_prev, engine='cfgrib') 

fname_next = directory_2 + f'{model}/{level}/{model}_{level}_{zone}_{date_next.year}{str(date_next.month).zfill(2)}{str(date_next.day).zfill(2)}000000.grib'
data_next = xr.open_dataset(fname_next, engine='cfgrib') 

if level !=  'PRECIP':
  if data_prev.step.values.shape[0] != 25:
    for param in params:
      param_values, valid_time_values = interpolate_in_time(data_prev[param].values, data_prev["valid_time"].values, 25, data_prev['time'].values)
      new_data_prev.assign_coords({param: param_values, 'valid_time': valid_time_values})
else:
  if data_prev.step.values.shape[0] !=24:
    for param in params:
      
      param_values, valid_time_values = interpolate_in_time(data_prev[param].values, data_prev["valid_time"].values, 24, data_prev['time'].values)
      new_data_prev.assign_coords({param: param_values, 'valid_time': valid_time_values})
if level !=  'PRECIP':
  if data_next.step.values.shape[0] != 25:
    for param in params:
      param_values, valid_time_values = interpolate_in_time(data_next[param].values, data_next["valid_time"].values, 25, data_next['time'].values)
      new_data_next.assign_coords({param: param_values, 'valid_time': valid_time_values})
else:
  if data_next.step.values.shape[0] !=24:
    for param in params:
      param_values, valid_time_values = interpolate_in_time(data_next[param].values, data_next["valid_time"].values, 24, data_next['time'].values)
      new_data_next.assign_coords({param: param_values, 'valid_time': valid_time_values})


data = data_prev.copy()
for param in params:
  data[param] = (data_prev[param] + data_next[param])/2

data = data.assign_coords({'time' : date})
data = data.assign_coords({'valid_time': data['valid_time'].values + np.timedelta64(1,'D')})

fname = f'/content/drive/My\ Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/models_2D_{zone}/{model}/{level}/{model}_{level}_{zone}_{date.year}{str(date.month).zfill(2)}{str(date.day).zfill(2)}000000.grib'

# !mv $fname '/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/Wrong_values'

new_fname = directory_2 + f'{model}/{level}/{model}_{level}_{zone}_{date.year}{str(date.month).zfill(2)}{str(date.day).zfill(2)}000000.nc'
data.to_netcdf(new_fname)

MissingDimensionsError: ignored

In [192]:
data_next

<xarray.Dataset>
Dimensions:            (latitude: 58, longitude: 80, step: 18, valid_time: 25)
Coordinates:
    time               datetime64[ns] 2017-08-27
  * step               (step) timedelta64[ns] 00:00:00 ... 1 days 00:00:00
    heightAboveGround  int64 2
  * latitude           (latitude) float64 51.9 51.8 51.7 51.6 ... 46.4 46.3 46.2
  * longitude          (longitude) float64 -5.842 -5.742 -5.642 ... 1.958 2.058
  * valid_time         (valid_time) datetime64[ns] 2017-08-27 ... 2017-08-28
Data variables:
    t2m                (step, latitude, longitude) float32 288.25806 ... 294.83862
    d2m                (step, latitude, longitude) float32 287.50638 ... 289.57635
    r                  (step, latitude, longitude) float32 95.26137 ... 72.040955
Attributes:
    GRIB_edition:            1
    GRIB_centre:             lfpw
    GRIB_centreDescription:  French Weather Service - Toulouse
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             French Weather Service - Toulouse
    history:                 2020-06-04T18:25:40 GRIB to CDM+CF via cfgrib-0....

In [200]:
data_next.assign({'t':(param_values.shape,param_values)})

TypeError: ignored

In [201]:
lala= xr.DataArray({'t':param_values}, coords={'valid_time': valid_time_values})

ValueError: ignored

In [202]:
lala_27

<xarray.Dataset>
Dimensions:            (latitude: 58, longitude: 80, step: 18)
Coordinates:
    time               datetime64[ns] ...
  * step               (step) timedelta64[ns] 00:00:00 ... 1 days 00:00:00
    heightAboveGround  int64 ...
  * latitude           (latitude) float64 51.9 51.8 51.7 51.6 ... 46.4 46.3 46.2
  * longitude          (longitude) float64 -5.842 -5.742 -5.642 ... 1.958 2.058
    valid_time         (step) datetime64[ns] ...
Data variables:
    t2m                (step, latitude, longitude) float32 ...
    d2m                (step, latitude, longitude) float32 ...
    r                  (step, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             lfpw
    GRIB_centreDescription:  French Weather Service - Toulouse
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             French Weather Service - Toulouse
    history:                 2020-06-04T18:34:01 GRIB to CDM+CF via cfgrib-0....

In [0]:
data = np.random.rand(4, 3)

locs = ['IA', 'IL', 'IN']

times = pd.date_range('2000-01-01', periods=4)
foo = xr.DataArray(data, coords=[times, locs], dims=['time', 'space'])

In [206]:
lala_27['valid_time'].values

array(['2017-08-27T00:00:00.000000000', '2017-08-27T03:00:00.000000000',
       '2017-08-27T04:00:00.000000000', '2017-08-27T05:00:00.000000000',
       '2017-08-27T07:00:00.000000000', '2017-08-27T08:00:00.000000000',
       '2017-08-27T10:00:00.000000000', '2017-08-27T11:00:00.000000000',
       '2017-08-27T12:00:00.000000000', '2017-08-27T14:00:00.000000000',
       '2017-08-27T16:00:00.000000000', '2017-08-27T17:00:00.000000000',
       '2017-08-27T18:00:00.000000000', '2017-08-27T19:00:00.000000000',
       '2017-08-27T20:00:00.000000000', '2017-08-27T21:00:00.000000000',
       '2017-08-27T23:00:00.000000000', '2017-08-28T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [209]:
lala_27['step'].values

array([             0, 10800000000000, 14400000000000, 18000000000000,
       25200000000000, 28800000000000, 36000000000000, 39600000000000,
       43200000000000, 50400000000000, 57600000000000, 61200000000000,
       64800000000000, 68400000000000, 72000000000000, 75600000000000,
       82800000000000, 86400000000000], dtype='timedelta64[ns]')

In [213]:
np.array(valid_time_values-valid_time_values[0], dtype='timedelta64[ns]')

array([             0,  3600000000000,  7200000000000, 10800000000000,
       14400000000000, 18000000000000, 21600000000000, 25200000000000,
       28800000000000, 32400000000000, 36000000000000, 39600000000000,
       43200000000000, 46800000000000, 50400000000000, 54000000000000,
       57600000000000, 61200000000000, 64800000000000, 68400000000000,
       72000000000000, 75600000000000, 79200000000000, 82800000000000,
       86400000000000], dtype='timedelta64[ns]')

In [215]:
lala_27['step'] = np.array(valid_time_values-valid_time_values[0], dtype='timedelta64[ns]')

ValueError: ignored

In [221]:
new_lala_27 = xr.DataArray({'t2m': (("latitude", "longitude"), param_values[0,:,:])},
                           coords={"time": lala_27['time'], 
                                   "latitude": lala_27["latitude"], "longitude": lala_27["longitude"],
                                   "valid_time": valid_time_values})

ValueError: ignored

In [229]:
lala_27["longitude"].values.shape

(80,)

In [0]:
x = xr.Dataset(
     {
         "t2m": (
             ("step","latitude", "longitude"),
             param_values,
         ),
         "precipitation": (("latitude", "longitude"), np.random.rand(58*80).reshape(58,80)),
     },
     coords={"latitude": lala_27["latitude"], "longitude": lala_27["longitude"],
             "step": np.array(valid_time_values-valid_time_values[0], dtype='timedelta64[ns]'),
             "valid_time": valid_time_values, "time": lala_27["time"]}
)

In [246]:
x.variables

Frozen({'t2m': <xarray.Variable (step: 25, latitude: 58, longitude: 80)>
array([[[288.25805664, 288.05102539, 287.79711914, ..., 290.87524414,
         290.76977539, 290.66821289],
        [288.37524414, 288.16430664, 287.89086914, ..., 290.85571289,
         290.77758789, 290.69946289],
        [288.50024414, 288.37524414, 288.21508789, ..., 290.64868164,
         290.56274414, 290.52758789],
        ...,
        [291.55102539, 291.56274414, 291.55493164, ..., 292.15649414,
         291.83227539, 291.83227539],
        [291.94555664, 291.93774414, 291.92211914, ..., 292.03540039,
         291.57836914, 291.61743164],
        [292.10571289, 292.11352539, 292.09790039, ..., 292.17602539,
         291.27758789, 290.71899414]],

       [[288.2074585 , 287.87802124, 287.55380249, ..., 290.62020874,
         290.54858398, 290.47827148],
        [288.32202148, 287.996521  , 287.65145874, ..., 290.56161499,
         290.52905273, 290.5043335 ],
        [288.48608398, 288.24130249, 287.9769897

In [242]:
lala_27

<xarray.Dataset>
Dimensions:            (latitude: 58, longitude: 80, step: 18, valid_time: 25)
Coordinates:
    time               datetime64[ns] 2017-08-27
  * step               (step) timedelta64[ns] 00:00:00 ... 1 days 00:00:00
    heightAboveGround  int64 2
  * latitude           (latitude) float64 51.9 51.8 51.7 51.6 ... 46.4 46.3 46.2
  * longitude          (longitude) float64 -5.842 -5.742 -5.642 ... 1.958 2.058
  * valid_time         (valid_time) datetime64[ns] 2017-08-27 ... 2017-08-28
Data variables:
    t2m                (step, latitude, longitude) float32 288.25806 ... 294.83862
    d2m                (step, latitude, longitude) float32 287.50638 ... 289.57635
    r                  (step, latitude, longitude) float32 95.26137 ... 72.040955
Attributes:
    GRIB_edition:            1
    GRIB_centre:             lfpw
    GRIB_centreDescription:  French Weather Service - Toulouse
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             French Weather Service - Toulouse
    history:                 2020-06-04T18:34:01 GRIB to CDM+CF via cfgrib-0....

In [8]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


In [0]:
lala = xr.open_dataset('/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/models_2D_NW/arpege/2m/arpege_2m_NW_20170826000000.nc')

In [0]:
lala_25 = xr.open_dataset('/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/models_2D_NW/arpege/2m/arpege_2m_NW_20170825000000.grib', engine='cfgrib')
lala_27 = xr.open_dataset('/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/models_2D_NW/arpege/2m/arpege_2m_NW_20170827000000.grib', engine='cfgrib')

In [0]:
def interpolate_in_time(array, num_timesteps):
  T = array.dims['step']
  H = array.dims['latitude']
  W = array.dims['longitude']
  array_times = array['valid_time']
  array_times_pd = pd.to_datetime(array_times)
  final_var = {}
  for var in array.data_vars:
    new_array = np.zeros([num_timesteps, H, W])
    
    real_date_pd = pd.to_datetime(real_date)
    hours = list(array_times_pd.hour)
    if array_times_pd[-1].day > real_date_pd.day:
      hours[-1] = 24
    for h in range(len(hours)):
      new_array[hours[h]] = array[h]
    for h in range(num_timesteps):
      if h not in hours:
        if h < hours[0]:
          new_array[h] = array[hours[0]]
        elif h > hours[-1]:
          new_array[h] = array[hours[-1]]
        else:
          m = np.argmin(abs(np.array(hours) - h))
          if hours[m] < h:
            new_array[h] = (hours[m+1] - h)/(hours[m + 1]- hours[m])*array[m] + (h-hours[m])/(hours[m+1]-hours[m])*array[m+1]
          else:
            new_array[h] = (hours[m] - h)/(hours[m]- hours[m-1])*array[m-1] + (h-hours[m-1])/(hours[m]-hours[m-1])*array[m]
  new_dates = np.array(pd.date_range(real_date_pd, periods=25, freq='H'))
  return new_array, new_dates

                        

In [257]:
for i in lala_27.data_vars:
  print(i)

t2m
d2m
r


In [0]:
new_fname = directory_2 + f'{model}/{level}/{model}_{level}_{zone}_{date.year}{str(date.month).zfill(2)}{str(date.day).zfill(2)}000000.nc'

In [0]:
new_fname = '/content/drive/My\ Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/models_2D_NW/arpege/2m/arpege_2m_NW_20170826000000.nc'
!rm $new_fname
# new_fname = f'/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/models_2D_{zone}/{model}/{level}/{model}_{level}_{zone}_{date.year}{str(date.month).zfill(2)}{str(date.day).zfill(2)}000000.nc'

# data.to_netcdf(new_fname, engine = 'netcdf4')

In [0]:
!wget "https://meteonet.umr-cnrm.fr/dataset/data/NW/weather_models/2D_parameters/NW_weather_models_2D_parameters_2017.tar.gz"

--2020-06-03 20:13:40--  https://meteonet.umr-cnrm.fr/dataset/data/NW/weather_models/2D_parameters/NW_weather_models_2D_parameters_2017.tar.gz
Resolving meteonet.umr-cnrm.fr (meteonet.umr-cnrm.fr)... 193.49.97.131
Connecting to meteonet.umr-cnrm.fr (meteonet.umr-cnrm.fr)|193.49.97.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8789487640 (8.2G) [application/x-gzip]
Saving to: ‘NW_weather_models_2D_parameters_2017.tar.gz’

NW_weather_models_2 100%[===================>]   8.19G  10.5MB/s    in 13m 46s 

2020-06-03 20:27:28 (10.1 MB/s) - ‘NW_weather_models_2D_parameters_2017.tar.gz’ saved [8789487640/8789487640]



In [0]:
!mv NW_weather_models_2D_parameters_2017.tar.gz "/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/tar_files/"

In [0]:
!tar -xvzf "/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/tar_files/NW_weather_models_2D_parameters_2017.tar.gz" -C "/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/tar_files/"

NW_weather_models_2D_parameters_2017/
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201701.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201702.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201703.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201704.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201705.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201706.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201707.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201708.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201709.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201710.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_201711.tar.gz
NW_weather_models_2D_parameters_2017/NW_weather_m

In [0]:
for i in [4,6,8]:
  print(i)
  if i < 10:
    fname = "/content/drive/My\ Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/tar_files/NW_weather_models_2D_parameters_2017/NW_weather_models_2D_parameters_20170" + str(i) + ".tar.gz"
  else:
    fname = "/content/drive/My\ Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/NW_rainfall_quality_code_2016/rainfall-diff-quality-code-NW-2016-" + str(i) + ".tar.gz"
  !tar -xzf $fname -C "/content/drive/My Drive/CS231N/Project/RainfallNowcasting/Radar/MeteoNet/tar_files/NW_weather_models_2D_parameters_2017/"

4
6
8
